# Mapa Interactivo

## Samuel Pedrosa

### O noso mapa amorsará a seguinte información:
### -Circuitos do mundo (Scrapping de Wikipedia)
### -Trazados dos circuitos de Formula 1 (arquivo.geojson, que contén os trazados dos circuitos de F1 da tempada 2021, e de algúns de tempada anteriores)
### -Fábricas dos equipos da tempada 2022 (arquivo .csv)
### -Arquivo geojson de países (Para ver os limites entre países)
### -Arquivo csv cos circuitos da que faltan no scraping,tempada 2021, xa que algún non aparece na Wikipedia

#### 1. Importación de librerías e carga de ficheiros

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import folium
import re
import folium
from folium.plugins import MarkerCluster

In [ ]:
file = 'C:/Users/samuel.pedrosapedros/Downloads/countries.geojson'
f1 = 'C:/Users/samuel.pedrosapedros/Downloads/f1_circuits.geojson'
constructores=pd.read_csv('C:/Users/samuel.pedrosapedros/Downloads/constructores.csv')
circuitos_2021=pd.read_csv('C:/Users/samuel.pedrosapedros/Downloads/circuitos.csv')

#### 2.Vamos scrapear os datos dos circutos da Wikipedia, para crear un DataFrame, onde almacenemos as cordenadas e o nome de cada circuito

In [ ]:
url = "https://es.wikipedia.org/wiki/Anexo:Circuitos_de_carreras"
paxina = requests.get(url)

In [ ]:
soup = BeautifulSoup(paxina.content, 'html.parser')

##### 2.1. Imos almacenar os nomes nunha lista, para despois metela nun DatFrame

In [ ]:
nomes_circuitos = []
for circuito in soup.find_all('th'):
    nomes_circuitos.append(circuito.text)

#### 2.2. Eliminamos os texto de encabezado da taboa, para quedar só cos nomes dos circuitos

In [ ]:
for i in range(0,5):
    nomes_circuitos.pop(0)
nomes_circuitos = nomes_circuitos[:226]
nomes_circuitos[:5]

##### 2.3. Limpamos os nomes para quedarnos soamente co texto

In [ ]:
nomes=[]
for i in range(0,226):
    nomes.append(nomes_circuitos[i].strip('\n'))
nomes[:3]

##### 2.4.Agora imos obter as coordenadas de latitude de cada circuito (O pop facémolo, para que só se nos garden as coordenadas decimais, que son as que queremos)

In [ ]:
lat=soup.find_all('span',{'class':'latitude'})
for i in range(0,226):
    lat.pop(i);
    i+2;

In [ ]:
len(lat)

In [ ]:
latitude=[]
for i in range(0,len(lat)):
    latitude.append(lat[i].text.strip(', '))
latitude[:3]

##### 2.5.Facemos o mesmo que ca latitude, pero neste caso coa lonxitude

In [ ]:
lon=soup.find_all('span',{'class':'longitude'})
for i in range(0,226):
    lon.pop(i);
    i+2;

In [ ]:
lonxitude=[]
for i in range(0,len(lon)):
    lonxitude.append(lon[i].text)
lonxitude[:3]

#### 3.Creamos un DataFrame, onde imos gardar os datos almacenados nas colunmas anteriores

##### 3.1.Creamos o DataFrame

In [ ]:
circuitos = pd.DataFrame(columns = ['Nome' , 'Latitude', 'Lonxitude'])

##### 3.2.Insertamos os datos das listas nas táboas correspondentes

In [ ]:
for i in range(0,226):
    circuitos=circuitos.append({'Nome' : nomes[i] , 'Latitude' : latitude[i], 'Lonxitude' : lonxitude[i]} , ignore_index=True)

In [ ]:
circuitos.head(5)

##### 3.3.Convertimos as táboas de Latitude e Lonxitude a float para que non dé erros á hora de tratalos numéricamente

In [ ]:
circuitos['Latitude'] = circuitos['Latitude'].astype(float)
circuitos['Lonxitude'] = circuitos['Lonxitude'].astype(float)
circuitos.info()

#### 4.Vamos observar os datos dos outros dous CSV.

In [ ]:
constructores.head(3)

In [ ]:
circuitos_2021.head(3)

In [ ]:
constructores.info()

In [ ]:
circuitos_2021.info()

##### 5.Pintamos o mapa

In [ ]:
#Establecemos o color para a capa dos países
style1 = {'fillColor': '#da33ff', 'lineColor': '#ff33f6'}
#Establecemos o color dos trazados dos circuitos
style2 = {'color':"yellow"}
#Como latitude e lonxitude de orixe, imos establecer a do Circuito de Monza, tamén coñecido como o templo da velocidade
latitude = "45.62222"
longitude = "9.28889"
#Creamos o mapa
m = folium.Map(location=[45.62222, 9.28889],zoom_start=8, width=1000, height=800)
#Creamos o tilelayer para cambiar o fondo a negro
tile_layer = folium.TileLayer(
    tiles="https://{s}.basemaps.cartocdn.com/rastertiles/dark_all/{z}/{x}/{y}.png",
    attr='&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors &copy; <a href="https://carto.com/attributions">CARTO</a>',
    max_zoom=19,
    name='Fondo Escuro',
    control=True,
    opacity=0.5
)
#Engadimos o fondo ao mapa
tile_layer.add_to(m)
#Creamos un clúster onde imos almacenar toda a información
marker_cluster=MarkerCluster(name="Circuitos e Fábricas").add_to(m)
#Cargamos os circuitos da Wikipedia
for i in range(0,226):
    url = 'https://nominatim.openstreetmap.org/search'
    busqueda =[circuitos.Latitude[i],circuitos.Lonxitude[i]]
    payload = {'q': busqueda ,'format':'json'}
    response = requests.get(url,params=payload)
    folium.Marker([circuitos.Latitude[i],circuitos.Lonxitude[i]],popup=(f'{circuitos.Nome[i]}'),icon=folium.Icon(color='red',icon='flag-checkered', prefix='fa')).add_to(marker_cluster)
#Cargamos a información dos constructores
for i in range(0,10):
    url = 'https://nominatim.openstreetmap.org/search'
    busqueda =[constructores.Latitude[i],constructores.Longitude[i]]
    payload = {'q': busqueda ,'format':'json'}
    response = requests.get(url,params=payload)
    folium.Marker([constructores.Latitude[i],constructores.Longitude[i]],popup=(f'{constructores.name[i]}'),icon=folium.Icon(color='green',icon='car', prefix='fa')).add_to(marker_cluster)
#Cargamos os datos dos outros circuitos
for i in range(0,7):
    url = 'https://nominatim.openstreetmap.org/search'
    busqueda =[circuitos_2021.lat[i],circuitos_2021.lng[i]]
    payload = {'q': busqueda ,'format':'json'}
    response = requests.get(url,params=payload)
    folium.Marker([circuitos_2021.lat[i],circuitos_2021.lng[i]],popup=(f'{circuitos_2021.name[i]}'),icon=folium.Icon(color='red',icon='flag-checkered', prefix='fa')).add_to(marker_cluster)
folium.GeoJson(file,name='paises',style_function=lambda x:style1).add_to(m)
folium.GeoJson(f1,name='circuitos_f1',style_function=lambda x:style2).add_to(m)
folium.LayerControl().add_to(m)
m.fit_bounds([latitude,longitude])
m

#### 6.En último lugar gardamos o mapa

In [ ]:
m.save('C:/Users/samuel.pedrosapedros/Downloads/mapa_interactivo.html')